# Stencil Numba Serial

In [1]:
! python --version

Python 3.7.3


In [2]:
import numba
print(numba.__version__)

0.41.0


In [4]:
import numpy
numpy.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']


## Código Python sequencial
 Somente para referência, conversão direta de Fortran

### Testes

In [15]:
import numpy as np
import sys
import time

def seq_iters(anew, aold, sizeEnd, niters, nsources, sources, energy, heat):
    for iters in range(0, niters, 2):
        for j in range(1, sizeEnd):
            for i in range(1, sizeEnd):
                anew[i,j] = aold[i,j]/2.0+  \
                (aold[i-1,j]+aold[i+1,j]+aold[i,j-1]+aold[i,j+1])/8.0
        for i in range(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy
        for j in range(1, sizeEnd):
            for i in range(1, sizeEnd):
                aold[i,j] = anew[i,j]/2.0+  \
                (anew[i-1,j]+anew[i+1,j]+anew[i,j-1]+anew[i,j+1])/8.0
        for i in range(0, nsources):
            aold[sources[i,0],sources[i,1]] += energy
    for j in range(1, sizeEnd):
        for i in range(1, sizeEnd):
            heat[0] = heat[0] + aold[i,j]

def seq():                 # 4800, 1, 500   
    n            = 4800    # nxn grid (4800,1,500)=1500; (100,1,10)=30
    energy       = 1       # energy to be injected per iteration
    niters       = 500     # number of iterations
    nsources     = 3       # sources of energy
    size         = n + 2
    sizeEnd      = n + 1
    heat         = np.zeros((1), np.float64)     # system total heat
    anew         = np.zeros((size, size), np.float64)
    aold         = np.zeros((size, size), np.float64)
    sources      = np.empty((3,2), np.int32)
    sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
    
    seq_iters(anew, aold, sizeEnd, niters, nsources, sources,  energy, heat)
    print("heat = %11.5f" %(heat[0]))

In [21]:
# t_seq = %timeit -n1 -r1 -o seq()
#     testes anteriores levaram mais de 4h

## Numba CPU Sequencial

Sequencial porém usa instruções de vetorização do processador

Testando o funcionamento no nó de login, antes de usar o arquivo de lote

### Testes

In [ ]:
%%timeit -n3 -r3
from numba import jit, config, prange
import numpy as np
import time

@jit(nopython=True, parallel=True)
def jit_iters_seq(anew,aold,sizeEnd,niters,nsources,sources,energy,heat):
    for iters in prange(0, niters):
        for j in prange(1, sizeEnd):
            for i in prange(1, sizeEnd):
                anew[i,j] = aold[i,j]/2.0+  \
                    (aold[i-1,j]+aold[i+1,j]+aold[i,j-1]+aold[i,j+1])/8.0
        for i in prange(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy
        for j in prange(1, sizeEnd):
            for i in prange(1, sizeEnd):
                aold[i,j] = anew[i,j]/2.0+  \
                (anew[i-1,j]+anew[i+1,j]+anew[i,j-1]+anew[i,j+1])/8.0
        for i in prange(0, nsources):
            aold[sources[i,0],sources[i,1]] += energy
    for j in prange(1, sizeEnd):
        for i in prange(1, sizeEnd):
            heat[0] = heat[0] + aold[i,j]

# Main
n            = 4800  # nxn grid (4800,1,500)=1500; (100,1,10)=30
energy       = 1.0   # energy to be injected per iteration [1.0]
niters       = 500   # number of iterations [500]
nsources     = 3     # sources of energy
size         = n + 2
sizeEnd      = n + 1
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
sources      = np.empty((3,2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = niters//2

t = time.time()
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1
jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources, energy, heat)
t = time.time() - t
print("Heat = %9.4f | Tempo = %8.4f | Thread count = %s" %  \
      (heat[0], t, config.NUMBA_NUM_THREADS))
# jit_iters_seq.parallel_diagnostics(level=4)

Heat = 1500.0000 | Tempo = 429.5704 | Thread count = 1


### Testes

In [2]:
from numba import jit, config, prange
import numpy as np
import time

@jit(nopython=True, parallel=True)
def jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources, energy, heat):
    for iters in range(0, niters, 2):
        for j in prange(1, sizeEnd):
            for i in prange(1, sizeEnd):
                anew[i,j] = ( aold[i,j]/2.0 +
                    (aold[i-1,j]+aold[i+1,j]+aold[i,j-1]+aold[i,j+1])/8.0 )
        for i in prange(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy
        for j in prange(1, sizeEnd):
            for i in prange(1, sizeEnd):
                aold[i,j] = ( anew[i,j]/2.0 +
                    (anew[i-1,j]+anew[i+1,j]+anew[i,j-1]+anew[i,j+1])/8.0 )
        for i in prange(0, nsources):
            aold[sources[i,0],sources[i,1]] += energy
    for j in prange(1, sizeEnd):
        for i in prange(1, sizeEnd):
            heat[0] = heat[0] + aold[i,j]


n            = 4800  # nxn grid (4800,1,500)=1500
energy       = 1.0   # energy to be injected per iteration
niters       = 5   # number of iterations
nsources     = 3     # sources of energy
size         = n + 2
sizeEnd      = n + 1
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
sources      = np.empty((3,2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

config.NUMBA_ENABLE_AVX = 1
config.DUMP_ASSEMBLY = 0
jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources, energy, heat)

### Testes

In [3]:
import numpy as np
from numba import jit, config, prange
from time import time
t = time()
config.DUMP_ASSEMBLY = 0
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1

@jit(nopython=True, parallel=True)
def jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources, energy, heat):
    for iters in range(0, niters):
        for j in prange(1, sizeEnd):
            for i in prange(1, sizeEnd):
                anew[i,j] = ( aold[i,j] * 0.5 + 
                     (aold[i-1,j]+aold[i+1,j]+aold[i,j-1]+aold[i,j+1]) * 0.125 )
        for i in prange(0, nsources) :
            anew[sources[i,0], sources[i,1]] += energy
        for j in prange(1, sizeEnd):
            for i in prange(1, sizeEnd):
                aold[i,j] = ( anew[i,j] * 0.5 +
                     (anew[i-1,j]+anew[i+1,j]+anew[i,j-1]+anew[i,j+1]) * 0.125 )
        for i in prange(0, nsources):
            aold[sources[i,0],sources[i,1]] += energy
    for j in prange(1, sizeEnd):
        for i in prange(1, sizeEnd):
            heat[0] = heat[0] + aold[i,j]

n            = 4800  # nxn grid (4800,1,500)=1500; (100,1,10)=30
energy       = 1.0   # energy to be injected per iteration [1.0]
niters       = 500   # number of iterations [500]
nsources     = 3     # sources of energy
size         = n + 2
sizeEnd      = n + 1
heat         = np.zeros((1), np.float64)     # system total heat
anew = aold  = np.zeros((size, size), np.float64)
sources      = np.empty((3,2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = niters // 2

jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources,
              energy, heat)
t = time() - t
print("Heat = %9.4f | Tempo = %8.4f | Thread count = %s"
      %(heat[0], t, config.NUMBA_NUM_THREADS))

Heat = 1500.0000 | Tempo =  99.5317 | Thread count = 1


### Testes

In [8]:
from numba import jit, config, prange
import numpy as np
import time
config.DUMP_ASSEMBLY = 0
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1

@jit(nopython=True, parallel=True)
def jit_iters_seq(anew, aold, sizeEnd, niters, nsources,
                  sources, energy, heat):
    for iters in prange(niters):
        for i in prange(1, sizeEnd):
            for j in prange(1, sizeEnd):
                anew[i,j] = ( aold[i,j] * 0.5 + (aold[i-1,j] +
                    aold[i+1,j] + aold[i,j-1] + aold[i,j+1]) * 0.125 )
        for i in prange(nsources) :
            anew[sources[i,0], sources[i,1]] += energy
        for i in prange(1, sizeEnd):
            for j in prange(1, sizeEnd):
                aold[i,j] = ( anew[i,j] * 0.5 + (anew[i-1,j] +
                    anew[i+1,j] + anew[i,j-1] + anew[i,j+1]) * 0.125 )
        for i in prange(nsources):
            aold[sources[i,0],sources[i,1]] += energy
    for i in prange(1, sizeEnd):
        for j in prange(1, sizeEnd):
            heat[0] += aold[i,j]

n            = 4800  # nxn grid (4800,1,500)=1500; (100,1,10)=30
energy       = 1     # energy to be injected per iteration [1.0]
niters       = 500   # number of iterations [500]
nsources     = 3     # sources of energy
size         = n + 2
sizeEnd      = n + 1
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
sources      = np.empty((3,2), np.int32)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = (niters+1) // 2

t0 = time.time()
jit_iters_seq(anew, aold, sizeEnd, niters, nsources,
              sources, energy, heat)
t0 = time.time() - t0
print("Heat = %0.4f | Tempo = %0.4f | Thread count = %s" %
      (heat[0], t0, config.NUMBA_NUM_THREADS))

Heat = 1500.0000 | Tempo = 23.2005 | Thread count = 1


### Testes

In [6]:
from time import time
tp = time()
import numpy as np
from numba import jit, config, prange

config.DUMP_ASSEMBLY = 0
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1

@jit(nopython=True, parallel=True)
def jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources, energy, heat):
    for iters in range(0, niters):
        anew[1:-1, 1:-1] =  ( aold[1:-1, 1:-1] / 2.0 + 
                            ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                              aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) / 8.0 )
        for i in range(0, nsources):
            anew[sources[i, 0], sources[i, 1]] += energy
        aold[1:-1, 1:-1] =  ( anew[1:-1, 1:-1] / 2.0 +
                            ( anew[2:  , 1:-1] + anew[ :-2, 1:-1] +
                              anew[1:-1, 2:  ] + anew[1:-1,  :-2] ) / 8.0 )
        for i in range(0, nsources):
            aold[sources[i,0],sources[i,1]] += energy
#    heat[0] = np.sum( aold[1:sizeEnd, 1:sizeEnd] )  # system total heat

n            = 4800    # nxn grid (4800,1,500)=1500; (4800,1,5)=12
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2
sizeEnd      = n + 1
heat         = np.zeros((1), np.float64)     # system total heat
anew = aold  = np.zeros((size,  size), np.float64)
sources      = np.empty((nsources, 2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = niters // 2

t = time()
jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources, energy, heat)
t = time() - t
tp = time() - tp
print("Heat = %0.4f | Tempo = %0.4f | TempoPyt = %0.4f | Thread count = %s" %
      (heat[0], t, tp, config.NUMBA_NUM_THREADS))

Heat = 0.0000 | Tempo = 70.0140 | TempoPyt = 70.0176 | Thread count = 1


# Rodando em um nó de execução
Grava o arquivo fonte:

### Testes

In [16]:
%%writefile st-nu-seq.py
from time import time
tp = time()
import numpy as np
from numba import jit, config, prange

config.DUMP_ASSEMBLY = 0
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1

@jit(nopython=True, parallel=True)
def jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources, energy, heat):
    for iters in range(0, niters):
        anew[1:-1, 1:-1] =  ( aold[1:-1, 1:-1] / 2.0 + 
                            ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                              aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) / 8.0 )
        for i in range(0, nsources):
            anew[sources[i, 0], sources[i, 1]] += energy
        aold[1:-1, 1:-1] =  ( anew[1:-1, 1:-1] / 2.0 +
                            ( anew[2:  , 1:-1] + anew[ :-2, 1:-1] +
                              anew[1:-1, 2:  ] + anew[1:-1,  :-2] ) / 8.0 )
        for i in range(0, nsources):
            aold[sources[i,0],sources[i,1]] += energy
    heat[0] = np.sum( aold[1:sizeEnd, 1:sizeEnd] )  # system total heat

n            = 4800    # nxn grid (4800,1,500)=1500; (4800,1,5)=12
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2
sizeEnd      = n + 1
heat         = np.zeros((1), np.float64)     # system total heat
anew = aold  = np.zeros((size,  size), np.float64)
sources      = np.empty((nsources, 2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = niters // 2

t = time()
jit_iters_seq(anew, aold, sizeEnd, niters, nsources, sources, energy, heat)
t = time() - t
tp = time() - tp
print("Heat = %0.4f | Tempo = %0.4f | TempoPyt = %0.4f | Thread count = %s" %
      (heat[0], t, tp, config.NUMBA_NUM_THREADS))

Overwriting st-nu-seq.py


In [17]:
%%writefile calctempo.sh
#!/bin/sh
time python st-nu-seq.py

Overwriting calctempo.sh


In [18]:
%%bash
sh calctempo.sh

Heat = 1500.0000 | Tempo = 68.2574 | TempoPyt = 68.7786 | Thread count = 1



real	1m8.993s
user	1m8.251s
sys	0m0.569s


### Copia o fonte para /scratch:

In [22]:
%%bash
a='st-nu-seq.py'
# diretórios já devem existir
s='/prj/ampemi/eduardo.miranda2/stnc/Numba'
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cp  $s/$a  $s/calctempo.sh  $d
ls -lh $d/$a

-rw-r--r-- 1 eduardo.miranda2 ampemi 1.8K Nov 11 17:41 /scratch/ampemi/eduardo.miranda2/stnc/Numba/st-nu-seq.py


In [23]:
%%bash
cd /scratch/ampemi/eduardo.miranda2/stnc/Numba
sh calctempo.sh

Heat = 1500.0000 | Tempo = 68.5121 | TempoPyt = 68.9992 | Thread count = 1



real	1m9.157s
user	1m8.506s
sys	0m0.560s


## Arquivo de submissão

### Testes

In [3]:
%%writefile st-nu-seq.srm
#!/bin/bash
# limites das filas (1,0 UA):
#   cpu_dev  : 20 min.,  1-4  nós, 1/1   tarefas em exec/fila máximo
#   cpu_small: 72 horas, 1-20 nós, 16/96 tarefas em exec/fila máximo

#SBATCH --ntasks=1             #Total de tarefas
#SBATCH -p cpu_small           #Fila (partition) a ser utilizada
#SBATCH -J stnuseq             #Nome do job, 8 caracteres
#SBATCH --time=00:05:00        #Tempo max. de execução
# #SBATCH --nodes=1              #Qtd de nós
# #SBATCH --ntasks-per-node=1    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)
# #SBATCH --exclusive            #Utilização exclusiva dos nós

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
nodeset -e $SLURM_JOB_NODELIST

#Entra no diretório de trabalho
cd /scratch/ampemi/eduardo.miranda2/stnc/Numba

#Modulos
module load anaconda3/2018.12

#Executavel
EXEC='python st-nu-seq.py'

#Dispara a execucao
echo '-- srun -------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS $EXEC
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '-- FIM --------------------------------'

Overwriting st-nu-seq.srm


## Envia para execução 3 vezes

In [25]:
%%bash
sbatch st-nu-seq.srm
sbatch st-nu-seq.srm
sbatch st-nu-seq.srm

Submitted batch job 772763
Submitted batch job 772764
Submitted batch job 772765
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [27]:
# verifica se já terminou
! squeue -n stf2seq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


## Mostra os arquivos de saída

In [28]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cat $d/slurm-772763.out
cat $d/slurm-772764.out
cat $d/slurm-772765.out

- Job ID: 772763
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1169
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- srun -------------------------------
$ srun -n  1 sh calctempo.sh
Heat = 1500.0000 | Tempo = 70.7601 | TempoPyt = 82.7772 | Thread count = 1

real	1m23.517s
user	1m9.141s
sys	0m0.522s
-- FIM --------------------------------
- Job ID: 772764
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1180
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
-- srun -------------------------------
$ srun -n  1 sh calctempo.sh
Heat = 1500.0000 | Tempo = 92.8287 | TempoPyt = 104.8471 | Thread count = 1

real	1m45.488s
user	1m31.355s
sys	0m0.670s
-- FIM --------------------------------
- Job ID: 772765
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1260
- diretorio onde sbatch foi chamado ($SLURM_SUB

## Versão final

Obs.: O código anterior acabou ficando mais lento que a versão MPI.

O código abaixo usa o mesmo kernel da versão MPI, para obter mais desempenho.

In [1]:
%%writefile st-nu-seq.py
import numpy as np
from numba import jit, config, prange
from time import time

config.DUMP_ASSEMBLY = 0
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1

@jit('(float64[:,:],float64[:,:])', nopython=True, parallel=True, nogil=True) 
def kernel_seq(anew, aold) :
    anew[1:-1, 1:-1] = ( aold[1:-1, 1:-1] * 0.5 + 
                       ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                         aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) * 0.125 )

    
n            = 4800    # nxn grid (4800,1,500)=1500; (4800,1,5)=12
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2
sizeEnd      = n + 1
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((size,  size), np.float64)
aold         = np.zeros((size,  size), np.float64)
sources      = np.empty((nsources, 2), np.int16)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = (niters + 1) // 2

t0 = time()
for iters in range(niters) :
    kernel_seq(anew, aold)
    for i in range(nsources) :
        anew[sources[i, 0], sources[i, 1]] += energy
    kernel_seq(aold, anew)
    for i in range(nsources) :
        aold[sources[i, 0], sources[i, 1]] += energy

heat[0] = np.sum( aold[1:-1, 1:-1] )  # system total heat
t0 = time() - t0

print("Heat = %0.4f | Tempo = %0.4f | Thread count = %s" %
      (heat[0], t0, config.NUMBA_NUM_THREADS))

Overwriting st-nu-seq.py


In [38]:
! python st-nu-seq-2.py

Heat = 1500.0000 | Tempo = 23.3059 | Thread count = 1


In [2]:
%%bash
a='st-nu-seq.py'
# diretórios já devem existir
s='/prj/ampemi/eduardo.miranda2/stnc/Numba'
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cp  $s/$a  $d
ls -lh $d/$a

-rw-r--r-- 1 eduardo.miranda2 ampemi 1.5K Nov 24 20:24 /scratch/ampemi/eduardo.miranda2/stnc/Numba/st-nu-seq.py


In [4]:
%%bash
sbatch st-nu-seq.srm

Submitted batch job 781495


In [5]:
! squeue -n stnuseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            781495 cpu_small  stnuseq eduardo. PD       0:00      1 (Priority)


In [8]:
! squeue -n stnuseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [9]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cat $d/slurm-781495.out

- Job ID: 781495
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1279
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
sdumont1279
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-seq.py
Heat = 1500.0000 | Tempo = 30.3982 | Thread count = 1
-- FIM --------------------------------


In [1]:
%%bash
sbatch st-nu-seq.srm
sbatch st-nu-seq.srm

Submitted batch job 788104
Submitted batch job 788105


In [2]:
! squeue -n stnuseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788104 cpu_small  stnuseq eduardo. PD       0:00      1 (Priority)
            788105 cpu_small  stnuseq eduardo. PD       0:00      1 (Priority)


In [1]:
! squeue -n stnuseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [2]:
%%bash
d='/scratch/ampemi/eduardo.miranda2/stnc/Numba'
cat $d/slurm-788104.out
cat $d/slurm-788105.out

- Job ID: 788104
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
sdumont1149
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-seq.py
Heat = 1500.0000 | Tempo = 30.5014 | Thread count = 1
-- FIM --------------------------------
- Job ID: 788105
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Numba
sdumont1149
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-seq.py
Heat = 1500.0000 | Tempo = 30.5266 | Thread count = 1
-- FIM --------------------------------
